# Molecular Building

MolPy’s molecular building tools are designed to be:

- **Composable** – work with `Atomistic`, `Frame` and `Box`
- **Engine‑agnostic** – you can later export to LAMMPS, OpenMM, …
- **Script‑friendly** – small functions you can call from Python/CLI

This page gives you a taste of the main pieces:

- Building polymers and assemblies
- Packing molecules into boxes

---

## 1. Building linear polymers

The `molpy.builder.polymer` helpers assemble polymers from **monomer templates**
and **reaction rules**. At the core you will usually see:

- `Monomer` / `Polymer` wrappers (in `molpy.core.wrappers`)
- A `ReacterConnector` that knows how to join ports
- An optional typifier (e.g. `OplsAtomisticTypifier`)



In [ ]:
from molpy.builder.polymer.connectors import ReacterConnector
from molpy.builder.polymer.linear import linear
from molpy.core.wrappers.monomer import Monomer
from molpy.typifier.atomistic import OplsAtomisticTypifier

# Assume you already built / loaded two monomer Atomistic structures: A, B
mono_A = Monomer(inner=A, name="A")
mono_B = Monomer(inner=B, name="B")
library = {"A": mono_A, "B": mono_B}

connector = ReacterConnector()  # chooses how ports react
typifier = OplsAtomisticTypifier()  # optional force‑field typing

poly = linear(
    sequence="ABABAB",  # monomer sequence
    library=library,
    connector=connector,
    typifier=typifier,
)

print(poly)  # Polymer wrapper around an Atomistic
inner = poly.inner  # underlying Atomistic structure
print("atoms:", len(list(inner.atoms)))
print("bonds:", len(list(inner.bonds)))


The `linear` builder takes care of:

- Validating that the sequence labels exist in your monomer library
- Connecting ports with the correct bond patterns
- (Optionally) re‑typifying once at the end

For more involved polymer workflows, see the tutorial:
`tutorials/polymer-building.ipynb`.

---

## 2. Packing molecules into a box

The `molpy.pack` module provides a high‑level interface to pack molecules
into a simulation box, using external packers such as **Packmol**.

The main entry point is `Molpack`:



In [ ]:
from pathlib import Path

import molpy as mp
from molpy.pack.constraints import BoxConstraint
from molpy.pack.molpack import Molpack

# 1. Single‑molecule template as a Frame
frame = mp.Frame()
frame["atoms", "x"] = [0.0, 1.0, 2.0]
frame["atoms", "y"] = [0.0, 0.0, 0.0]
frame["atoms", "z"] = [0.0, 0.0, 0.0]
frame["atoms", "element"] = ["C", "C", "C"]
frame.metadata["box"] = mp.Box.cubic(20.0)

# 2. Define packing target: 100 copies inside the box
workdir = Path("packing")
packer = Molpack(workdir=workdir, packer="packmol")

constraint = BoxConstraint(box=frame.metadata["box"])
packer.add_target(frame, number=100, constraint=constraint)

# 3. Run the optimization (requires Packmol installed)
result = packer.optimize(max_steps=1000, seed=42)

print("Packed atoms:", result["atoms"].nrows)
print("Box:", result.metadata.get("box"))


High‑level flow:

1. You provide one or more **template frames** (single molecules).
2. You define **targets** (how many copies, where to place them, constraints).
3. `Molpack` writes intermediate structure files and calls the selected backend.
4. You get back a packed `Frame` ready for file IO or simulation setup.

If you want more control, you can use the lower‑level `Packer` classes directly
or write your own packer that implements the same interface.

---

## 3. From builders/packers to simulation input

The typical modeling pipeline looks like this:

1. Use **builder** (and sometimes **reacter**) to obtain an `Atomistic` structure.
2. Convert to a `Frame` (+ `Box`) if needed (for IO/engine).
3. Optionally pack many copies into a larger box using `molpy.pack`.
4. Use the tools described in `user-guide/simulation-setup.md` to export
   LAMMPS/OpenMM input files.

The goal is that you always move between a small set of types:

- Build & chemistry: `Atomistic` (+ wrappers like `Monomer`, `Polymer`)
- Spatial packing: `Frame` + `Box`
- Simulation: `Frame` + force‑field + engine‑specific writers

